# Scalable K-means++

## Outline of Background

1. ***K*-means** remains one of the most popular data processing algorithms. However a proper initialization is crucial for reciveing successful results. 

2. ***K*-means++** algorithm achived the goal of finding proper initialization with downside of its inherent sequentical nature, which limits its efficiency ( *O(n)* ) and applicability to big massive datasets.

3. The paper *Scalable* proposed the ***k*-means||** algorithm with efficiency ( *O(log n)* ), which forms ***k*-means++** in both sequential and parallel settings.

## Algorithm / Pseudocode

### Notations

Let $X =\{x_1,...x_n\}$ be the set of points in $d$-dimensional Euclidean space, and let $k$ be a positive integer specifying the number of clusters. Let ||$x_i$ - $x_j$ || denote the Euclidean distance between $x_i$ and $x_j$. For a point $x$ and a subset $Y \subseteq X$ of points, the distance is defined as $d(x,Y) = min_{y \in Y} ||x - y||$. For a subset $Y \subseteq X$ of points, let its \text{centroid} be given by
\begin{equation*}
\begin{split}
    \text{centroid}(Y) = \frac{1}{|Y|}\sum_{y \in Y} x
\end{split}
\end{equation*}
Let $C =\{c_1,...c_k\}$ be the ser of points and let $Y \subseteq X$. We define the \text{cost} of $Y$ with respect to $C$ as
\begin{equation*}
\begin{split}
    \phi_Y(C) = \sum_{y \in Y} d^2(y,C) = \sum_{y \in Y} \min_{i=1,...,k}||y-c_i||^2
\end{split}
\end{equation*}


### $k$-means++($k$) initialization
1. $C \leftarrow$ sample a point uniformlt at random from $X$ 
2. **while** $|C|<k$ **do**
  - Sample $x \in X$ with probability $\frac{d^2(x,C)}{\phi_X(C)}$
  - $C \leftarrow C \cup \{x\}$ 
3. **end while**

### $k$-means||($k,l$) initialization
1. $C \leftarrow$ sample a point uniformlt at random from $X$ 
2. $\psi \leftarrow \phi_X(C)$
3. **for** $O$(log$\psi$) times **do**
  - $C' \leftarrow$ sample each point $x \in X$ independently with probability $p_x = \frac{l \cdot d^2(x,C)}{\phi_X(C)}$	
  - $C \leftarrow C \cup C'$ 
4. **end for** 
5. For $x \in C$, set $w_x$ to be the number of points in $X$ closer to $x$ than any point in $C$ 
6. Recluster the weighted points in $C$ into $k$ clusters

## Draft of unit test

**Will verify the code correctness using following tests:**

1. Test the distance using both ***k*-means++** and ***k*-means||** algorithms to verify the results
2. Test distance function using examples in terms of: 
  - symmetry
  - zero
  - non-negativity 
3. Bechmark test compare the efficiency of ***k*-means++** and ***k*-means||** algorithms.

## Optimization Strategies

1. Using alternative method to replace for loop in Python
2. Will try on large datasets, if still take too long. Could try to use other languages to write the looping part

## Data simulation

Simulating sample data from three bivariate normal distribution.

\begin{eqnarray*}
\begin{pmatrix}x_{1}\\
x_{2}
\end{pmatrix} & \sim & N\left[\left(\begin{array}{c}
0\\
1
\end{array}\right),\left(\begin{array}{ccc}
1 & 0.5\\
0 & 2
\end{array}\right)\right]\\
\begin{pmatrix}y_{1}\\
y_{2}
\end{pmatrix} & \sim & N\left[\left(\begin{array}{c}
3\\
7
\end{array}\right),\left(\begin{array}{ccc}
1 & 0.33\\
0.33 & 1
\end{array}\right)\right]\\
\begin{pmatrix}z_{1}\\
z_{2}
\end{pmatrix} & \sim & N\left[\left(\begin{array}{c}
-8\\
2
\end{array}\right),\left(\begin{array}{ccc}
2 & 0.66\\
0.66 & 2
\end{array}\right)\right]\\
\end{eqnarray*}



In [5]:
%%file DataSimulation.py

# Simulated "Real" Data Set
#!/usr/bin/python
import os
import sys
import glob
import random
import numpy as np
import pandas as pd

def DataSimulation(n):
    mean0 = [0, 1]
    cov0 = [[1, 0.5], [0.5, 2]]
    data0 = np.random.multivariate_normal(mean0, cov0, n)
    data0 = np.hstack((data0, np.ones((data0.shape[0],1))))

    mean1 = [3, 7]
    cov1 = [[1, 0.33], [0.33, 1]]
    data1 = np.random.multivariate_normal(mean1, cov1, n)
    data1 = np.hstack((data1, np.ones((data1.shape[0],1)) * 2))

    mean2 = [-8, 2]
    cov2 = [[2, 0.66], [0.66, 2]]
    data2 = np.random.multivariate_normal(mean2, cov2, n)
    data2 = np.hstack((data2, np.ones((data2.shape[0],1)) * 3))

    data = np.vstack((data0, data1, data2))
    np.random.shuffle(data)
    print (data.shape)
    return data

DataSimulation(50)

(150, 3)


array([[  5.57524633e-01,   1.79411743e+00,   1.00000000e+00],
       [ -6.94784243e+00,   5.22336967e+00,   3.00000000e+00],
       [ -5.21833773e-01,   2.17736178e+00,   1.00000000e+00],
       [  7.11255766e-01,   5.70181667e+00,   2.00000000e+00],
       [  1.57342873e+00,   1.39023547e+00,   1.00000000e+00],
       [  1.36293171e+00,   5.81830139e+00,   2.00000000e+00],
       [ -8.44499307e-01,   4.87658911e-01,   1.00000000e+00],
       [ -5.91854044e+00,   2.76379068e+00,   3.00000000e+00],
       [ -2.82598978e-01,   8.61343498e-02,   1.00000000e+00],
       [  2.83049640e+00,   6.35305620e+00,   2.00000000e+00],
       [ -6.91225255e+00,   1.19906541e+00,   3.00000000e+00],
       [ -3.16911985e-01,   2.64092125e+00,   1.00000000e+00],
       [ -4.34184185e-01,  -4.62357958e-01,   1.00000000e+00],
       [ -8.38795400e-01,   1.97600107e+00,   1.00000000e+00],
       [ -9.33366176e+00,   3.91790189e-01,   3.00000000e+00],
       [  3.12248734e+00,   7.79461164e+00,   2.0000000

## Algorithm comparison

For algorithm comparison, I will using different sizes of same sample data to clustering using orginal k-means, k-means++, k-means||. Test their result and compare the efficiency.

## Code